# Análisis y Predicción de Ventas en una Tienda de Retail (Core)

### Parte III: Transformación y Análisis Avanzado de Datos con Pandas

Este notebook realiza un análisis avanzado en el dataset de ventas retail proporcionado.
Incluye transformación, agrupación y análisis personalizado utilizando Pandas.

In [2]:
# Paso 1: Preparación del Entorno
import pandas as pd

# Cargar el dataset de ventas retail
file_path = 'data/retail_sales_dataset.csv'
df = pd.read_csv(file_path)
df['Date'] = pd.to_datetime(df['Date'])  # Convertir la columna Date a formato datetime para análisis temporal
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


## Paso 2: Transformación de Datos

In [3]:
# Crear nuevas columnas para el análisis

# Calcular ingreso total por venta
df['Total Revenue'] = df['Quantity'] * df['Price per Unit']

# Normalización de ventas
df['Normalized Revenue'] = (df['Total Revenue'] - df['Total Revenue'].mean()) / df['Total Revenue'].std()

# Clasificación de ventas: Alta, Media, Baja según percentiles
def classify_sales(revenue):
    if revenue >= df['Total Revenue'].quantile(0.75):
        return 'Alta'
    elif revenue >= df['Total Revenue'].quantile(0.25):
        return 'Media'
    else:
        return 'Baja'

df['Sales Category'] = df['Total Revenue'].apply(classify_sales)
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Total Revenue,Normalized Revenue,Sales Category
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,150,-0.546431,Media
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,1000,0.971433,Alta
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,30,-0.760718,Baja
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,500,0.078572,Media
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,100,-0.635717,Media


## Paso 3: Agrupación y Agregación

In [4]:
# Agrupación por 'Product Category' y 'Date' para obtener estadísticas descriptivas

product_store_group = df.groupby(['Product Category', df['Date'].dt.month])

# Aplicar funciones de agregación
agg_stats = product_store_group['Total Revenue'].agg(['sum', 'mean', 'count', 'min', 'max', 'std', 'var'])
print('\nEstadísticas descriptivas por categoría de producto y mes:')
print(agg_stats)



Estadísticas descriptivas por categoría de producto y mes:
                         sum        mean  count  min   max         std  \
Product Category Date                                                    
Beauty           1     13930  535.769231     26   25  2000  658.818173   
                 2     14035  539.807692     26   25  2000  697.754227   
                 3     10545  502.142857     21   25  1500  482.914515   
                 4     11905  410.517241     29   25  2000  519.650200   
                 5     12450  444.642857     28   30  2000  571.555591   
                 6     10995  439.800000     25   25  2000  512.246848   
                 7     16090  595.925926     27   30  2000  626.063767   
                 8      9790  407.916667     24   25  1200  451.730770   
                 9      6320  316.000000     20   25  2000  505.963385   
                 10    15355  495.322581     31   25  2000  552.575689   
                 11     9700  388.000000     25   25

## Paso 4: Análisis Personalizado con apply

In [5]:
# Calcular la desviación de cada venta respecto a la media de su grupo

def deviation_from_mean(x):
    return x - x.mean()

df['Revenue Deviation'] = df.groupby('Product Category')['Total Revenue'].transform(deviation_from_mean)
df[['Product Category', 'Total Revenue', 'Revenue Deviation']].head()

,Product Category,Total Revenue,Revenue Deviation
0,Beauty,150,-317.475570
1,Clothing,1000,556.752137
2,Electronics,30,-428.786550
3,Clothing,500,56.752137
4,Beauty,100,-367.475570
